In [2]:
import torch
import torch.utils.data as data_utils
from torch.nn import Module
from torch.nn import Conv2d, Conv1d
from torch.nn import Linear
from torch.nn import MaxPool2d, MaxPool1d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten, tensor
import pandas as pd
import numpy as np
from torch.optim import Adam
import time


In [3]:
df_input = pd.read_csv('/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/input.csv', index_col=0)
df_output = pd.read_csv('/content/drive/MyDrive/Gdrive/startup/Novus Sentry/src/output.csv', index_col=0)

In [4]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
# define the train and val splits
TRAIN_SPLIT = 0.75
VAL_SPLIT = 1 - TRAIN_SPLIT
# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def create_test_train_data(df_input, df_output):
  train_x = df_input.sample(frac=0.8)
  train_y = df_output.loc[train_x.index]

  test_x = df_input.drop(train_x.index)
  test_y = df_output.loc[test_x.index]

  return train_x, train_y, test_x, test_y

In [6]:
train_x, train_y, test_x, test_y = create_test_train_data(df_input, df_output)

In [7]:
train_target = tensor(train_y.iloc[:,-1].values)
train = tensor(train_x.values) 
train_tensor = data_utils.TensorDataset(train, train_target) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)

test_target = tensor(test_y.iloc[:,-1].values)
test = tensor(test_x.values) 
test_tensor = data_utils.TensorDataset(test, test_target) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = BATCH_SIZE, shuffle = True)

In [8]:
class LeNet(Module):

	def __init__(self):
		# call the parent constructor
		super(LeNet, self).__init__()
		# initialize first set of CONV => RELU => POOL layers
		self.conv1 = Conv1d(in_channels=1, out_channels=1, kernel_size=5)
		self.relu1 = ReLU()
		self.maxpool1 = MaxPool1d(kernel_size=2, stride= 2)
		# initialize second set of CONV => RELU => POOL layers
		self.conv2 = Conv1d(in_channels=20, out_channels=50, kernel_size=5)
		self.relu2 = ReLU()
		self.maxpool2 = MaxPool1d(kernel_size=2, stride= 2)
		# initialize first (and only) set of FC => RELU layers
		self.fc1 = Linear(in_features=800, out_features=500)
		self.relu3 = ReLU()
		# initialize our softmax classifier
		self.fc2 = Linear(in_features=500, out_features=1)
		self.logSoftmax = LogSoftmax(dim=1)
    
	def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
		x = self.conv1(x)
		x = self.relu1(x)
		x = self.maxpool1(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv2(x)
		x = self.relu2(x)
		x = self.maxpool2(x)
		# flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
		x = flatten(x, 1)
		x = self.fc1(x)
		x = self.relu3(x)
		# pass the output to our softmax classifier to get our output
		# predictions
		x = self.fc2(x)
		output = self.logSoftmax(x)
		# return the output predictions
		return output

In [9]:
trainSteps = len(train_loader.dataset) // BATCH_SIZE
valSteps = len(test_loader.dataset) // BATCH_SIZE

In [10]:
print("[INFO] initializing the LeNet model...")
model = LeNet().to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = torch.nn.NLLLoss()
# initialize a dictionary to store training history
H = {
	"train_loss": [],
	"train_acc": [],
	"val_loss": [],
	"val_acc": []
}
# measure how long training is going to take
print("[INFO] training the network...")
startTime = time.time()

[INFO] initializing the LeNet model...
[INFO] training the network...


In [11]:
for e in range(0, EPOCHS):
	# set the model in training mode
	model.train()
	# initialize the total training and validation loss
	totalTrainLoss = 0
	totalValLoss = 0
	# initialize the number of correct predictions in the training
	# and validation step
	trainCorrect = 0
	valCorrect = 0
	# loop over the training set
	for (x, y) in train_loader:
		# send the input to the device
		(x, y) = (x.to(device), y.to(device))
		# perform a forward pass and calculate the training loss
		pred = model(x)
		loss = lossFn(pred, y)
		# zero out the gradients, perform the backpropagation step,
		# and update the weights
		opt.zero_grad()
		loss.backward()
		opt.step()
		# add the loss to the total training loss so far and
		# calculate the number of correct predictions
		totalTrainLoss += loss
		trainCorrect += (abs(pred-y)/len(y)).type(
			torch.float).sum().item()

RuntimeError: ignored